# Sentiment Analysis in the Browser

In this notebook, we will show how to create a `.air` file to perform sentiment analysis in the browser using a neural network.  To do this, we will utilize the IMDB Movie Reviews dataset to build the initial model and then package the model using the `aisquared` Python SDK.

## Dependencies

For this notebook, the following dependencies are required:

- `aisquared`

This package is available on [pypi](https://pypi.org) via `pip`.

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import tensorflow as tf
import pandas as pd
import aisquared

## Model Creation

Now that the required packages have been installed and imported, it is time to create the sentiment analysis model.  To do this, we have to first load and preprocess the data, create the model, and then package the model in the `.air` format.  The following cells will go through an in-depth explanation of each of the steps in this process.

In [ ]:
df = pd.read_csv('IMDB Dataset.csv')
df.head()

In [ ]:
# Load the data and tokenize

tokenizer = tf.keras.preprocessing.text.Tokenizer(10000, oov_token = 1)
tokenizer.fit_on_texts(df.review)
vocab = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(df.review)
sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, 256, padding = 'pre', truncating = 'post')

labels = df.sentiment.apply(lambda x : {'positive' : 0, 'negative' : 1}[x]).values

x_train, x_test, y_train, y_test = train_test_split(sequences, labels, train_size = 0.7)
del vocab[1]

In [ ]:
# Create the model

input_layer = tf.keras.layers.Input(sequences.shape[1:])
embedding_layer = tf.keras.layers.Embedding(
    10000,
    4
)(input_layer)
x = tf.keras.layers.Flatten()(embedding_layer)
for _ in range(5):
    x = tf.keras.layers.Dense(1000, activation = 'relu')(x)
output_layer = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)

model = tf.keras.models.Model(input_layer, output_layer)
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

In [ ]:
# Compile the model
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

# Train the model with the sparsification callback
model.fit(
    x_train,
    y_train.reshape(-1,1),
    epochs = 20,
    batch_size = 512,
    validation_split = 0.2,
    verbose = 2,
    callbacks = None
)

In [ ]:
# Check model performance
preds = (model.predict(x_test) >= 0.5).astype(int)
print('Model Performance on Test Data:')
print('\n')
print(confusion_matrix(y_test, preds))
print(classification_report(y_test, preds))

# Save the model
model.save('SentimentClassifier.h5')

## Package the Model

Now that the model has been created, we can package the model into a single `.air` file that enables integration into the browser.

To perform this packaging, we will be utilizing the `aisquared` package `DocumentPredictor` class.

In [ ]:
harvester = aisquared.config.harvesting.InputHarvester()

preprocessor = aisquared.config.preprocessing.text.TextPreprocesser(
    [
        aisquared.config.preprocessing.text.RemoveCharacters(),
        aisquared.config.preprocessing.text.ConvertToCase(lowercase = True),
        aisquared.config.preprocessing.text.Tokenize(),
        aisquared.config.preprocessing.text.ConvertToVocabulary(vocabulary = vocab, oov_character = 1, start_character = 0),
        aisquared.config.preprocessing.text.PadSequences(length = 256, pad_location = 'pre', truncate_location = 'post')
    ]
)   
analytic = aisquared.config.analytic.LocalModel('SentimentClassifier.h5', 'text')

postprocessor = aisquared.config.postprocessing.BinaryClassification(['positive', 'negative'], 0.5)

renderer = aisquared.config.rendering.DocumentRendering(include_probability = True)

feedback = aisquared.config.feedback.BinaryFeedback(['positive', 'negative'])

aisquared.config.ModelConfiguration(
    'SentimentClassifier',
    harvester,
    preprocessor,
    analytic,
    postprocessor,
    renderer,
    feedback).compile(dtype = 'float16')